In [1]:
%pip install sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize


In [3]:
df = pd.read_csv("spotify_prepared_for_agents.csv")

df.shape


(1000, 12)

In [4]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(df["clean_content"].tolist(), show_progress_bar=True)

df["embedding"] = list(embeddings)


/Users/suruthe/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/suruthe/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Batches: 100%|██████████| 32/32 [00:52<00:00,  1.63s/it]


In [5]:
# Prepare weighted embeddings for clustering: We want important reviews to influence clusters more.
X = np.vstack(df["embedding"].values)

# Apply final_weight
X_weighted = X * df["final_weight"].values.reshape(-1, 1)

# Normalize for clustering stability
X_weighted = normalize(X_weighted)


In [6]:
# KMeans clustering: Choose Number of Themes (Broad = 6)
k = 6
kmeans = KMeans(n_clusters=k, random_state=42)
df["theme_id"] = kmeans.fit_predict(X_weighted)


/Users/suruthe/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/suruthe/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/suruthe/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/suruthe/Library/Python/3.9/lib/python/site-packages/sklearn/cluster/_kmeans.py:237: RuntimeWarning: divide by zero encountered in matmul
  current_pot = closest_dist_sq @ sample_weight
/Users/suruthe/Library/Python/3.9/lib/python/site-packages/sklearn/cluster/_kmeans.py:237: RuntimeWarning: overflow encountered in matmul
  current_pot = closest_dist_sq @ sample_weight
/Users/suruthe/Library/Python/3.9/lib/python/site-packages/sklearn/cluster/_kmeans.py:237: RuntimeWarning: invalid value encountered in matmul
  current_pot = cl

In [7]:
for i in range(k):
    print(f"\n--- 🎯 Theme {i} ---")

    print("\n🔹 Representative Samples (Random):")
    rep_samples = (
        df[df["theme_id"] == i]
        .sample(3, random_state=42)
    )
    for text in rep_samples["content"].values:
        print("-", text[:200])

    print("\n🔥 High-Impact Samples:")
    top_samples = (
        df[df["theme_id"] == i]
        .sort_values("final_weight", ascending=False)
        .head(3)
    )
    for _, row in top_samples.iterrows():
        print(f"[{row['score']}⭐ | Weight {row['final_weight']:.2f}] - {row['content'][:200]}")



--- 🎯 Theme 0 ---

🔹 Representative Samples (Random):
- Last update has caused issues for Android 13. Open album or playlist and play song, navigate to Android main phone home screen (keep playing music), go back to Spotify and Spotify re-launches the home
- I love Spotify for playing music, but for the love of all that is holy please give us an option to disable auto play in my vehicle! Every time I connect via Bluetooth, it starts playing music automati
- What is going on with spotify? It drops after one song. I have tried to uninstall it and reinstall it and still having the same issue. It seems like Spotify has an issue at least three four times a ye

🔥 High-Impact Samples:
[1⭐ | Weight 5.55] - Absolutely terrible on mobile. I can't actually play the songs I want to because it just plays a different song when I try. Otherwise, if I try to play a song in my playlist, it just gives me a previe
[1⭐ | Weight 5.06] - *I cant stress this enough spotify* You need to show the album covers

In [8]:
theme_map = {
    0: "Playback Reliability",
    1: "Navigation & Home Feed",
    2: "Library & Playlist Control",
    3: "Free vs Premium Friction",
    4: "UI & Content Surfaces",
    5: "Performance & Media Issues"
}

df["theme_label"] = df["theme_id"].map(theme_map)


In [9]:
import numpy as np

theme_centroids = {}

for theme in df["theme_label"].unique():
    emb = np.vstack(df[df["theme_label"] == theme]["embedding"].values)
    theme_centroids[theme] = emb.mean(axis=0)


In [10]:
from sklearn.metrics.pairwise import cosine_similarity

def assign_themes(embedding, centroids, threshold=0.35):
    labels = []
    for theme, centroid in centroids.items():
        sim = cosine_similarity([embedding], [centroid])[0][0]
        if sim >= threshold:
            labels.append(theme)
    return labels

df["assigned_themes"] = df["embedding"].apply(
    lambda x: assign_themes(x, theme_centroids)
)


In [12]:
df_exploded = df.explode("assigned_themes")


In [13]:
theme_distribution = (
    df_exploded["assigned_themes"]
    .value_counts(normalize=True)
)

print(theme_distribution)


assigned_themes
Performance & Media Issues    0.168736
UI & Content Surfaces         0.167205
Navigation & Home Feed        0.166865
Library & Playlist Control    0.166865
Free vs Premium Friction      0.165334
Playback Reliability          0.164994
Name: proportion, dtype: float64


In [14]:
df.to_csv("spotify_reviews_with_base_themes.csv", index=False)
df_exploded.to_csv("spotify_reviews_multilabel.csv", index=False)
